<a href="https://colab.research.google.com/github/anyuanay/INFO213/blob/main/INFO213_Week10_clustering_kmeans_lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INFO 213: Data Science Programming 2
___

### Week 10: Clustering and K-Means


**Agenda:**
- Definition and significance of clustering in unsupervised learning.
- Differentiating between supervised and unsupervised learning.
- Detailed explanation of the k-means algorithm.
- How the algorithm initializes centroids and iteratively assigns data points to clusters.


# What is Clustering? How is clustering different from classification?

Here we will examine a class of unsupervised machine learning models: clustering algorithms.

- Clustering algorithms seek to learn, from the properties of the data, an optimal division or discrete labeling of groups of points.

- Many clustering algorithms are available in Scikit-Learn and elsewhere, but perhaps the simplest to understand is an algorithm known as *k-means clustering*, which is implemented in ``sklearn.cluster.KMeans``.
- Unlike classification, clusturing doesn't need to learn predefined labels for data.

We begin with the standard imports:

```python
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
```

## What is the K-Means clustering algorithm? How does the K-Means algorithm work?

The *k*-means algorithm searches for a pre-determined number of clusters within an unlabeled multidimensional dataset.
It accomplishes this using a simple conception of what the optimal clustering looks like:

- The "cluster center" is the arithmetic mean of all the points belonging to the cluster.
- Each point is closer to its own cluster center than to other cluster centers.

Those two assumptions are the basis of the *k*-means model.
We will soon dive into exactly *how* the algorithm reaches this solution, but for now let's take a look at a simple dataset and see the *k*-means result.

First, let's generate a two-dimensional dataset containing four distinct blobs.
To emphasize that this is an unsupervised algorithm, we will leave the labels out of the visualization

```python
from sklearn.datasets import make_blobs
X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);
```

By eye, it is relatively easy to pick out the four clusters.
The *k*-means algorithm does this automatically, and in Scikit-Learn uses the typical estimator API:

```python
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, n_init=10)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
```

Let's visualize the results by plotting the data colored by these labels.
We will also plot the cluster centers as determined by the *k*-means estimator:

```python
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);
```

- The good news is that the *k*-means algorithm (at least in this simple case) assigns the points to clusters very similarly to how we might assign them by eye.
- But you might wonder how this algorithm finds these clusters so quickly! After all, the number of possible combinations of cluster assignments is exponential in the number of data points—an exhaustive search would be very, very costly.
- Fortunately for us, such an exhaustive search is not necessary: instead, the typical approach to *k*-means involves an intuitive iterative approach known as *expectation–maximization*.

## What is expectation-maximazation? How is expectaion-maximazation used in the k-means algorithm?

- Expectation–maximization (E–M) is a powerful algorithm that comes up in a variety of contexts within data science.
- *k*-means is a particularly simple and easy-to-understand application of the algorithm, and we will walk through it briefly here.
- In short, the expectation–maximization approach here consists of the following procedure:

1. Guess some cluster centers
2. Repeat until converged
   1. *E-Step*: assign points to the nearest cluster center
   2. *M-Step*: set the cluster centers to the mean

- Here the "E-step" or "Expectation step" is so-named because it involves updating our expectation of which cluster each point belongs to.
- The "M-step" or "Maximization step" is so-named because it involves maximizing some fitness function that defines the location of the cluster centers—in this case, that maximization is accomplished by taking a simple mean of the data in each cluster.

- The literature about this algorithm is vast, but can be summarized as follows:
    - under typical circumstances, each repetition of the E-step and M-step will always result in a better estimate of the cluster characteristics.

## How is the k-means algorithm implemented?
In practice, the k-means algorithm is summarized by the following four steps:
1. Randomly pick $k$ centroids from the examples as initial cluster centers.
2. Assign each example to the nearest centroid, $\mu^{(j)}, j\in\{1..k\}$.
3. Move the centroids to the center of the examples that were assigned to it.
4. Repeat steps 2 and 3 until the cluster assignments do not change or a user-defined tolerance or maximum number of iterations is reached.

- The similarity between objects is defined by the squared Euclidean distance between two points $x$ and $y$ in $m$-dimensional space:
\begin{equation}
 d(x, y)^2 = \Sigma_j^m(x_j-y_j)^2
\end{equation}

## Manual Example
1. Data points: $\{a=(1, 2), b=(2, 1), c=(3, 4), d=(4, 6), e=(5, 3)\}$ and $K = 2$
2. Randomly pick 2 centroids: $a=(1, 2)$ and $b=(2, 1)$
3. Compute the distances from other points to the centroids:
 - $d(c,a)^2 = (3-1)^2+(4-2)^2=8$
 - $d(c,b)^2 = (3-2)^2+(4-1)^2=10$
 - $d(d,a)^2 = (4-1)^2+(6-2)^2=25$
 - $d(d,b)^2 = (4-2)^2+(6-1)^2=29$
 - $d(e,a)^2 = (5-1)^2+(3-2)^2=17$
 - $d(e,b)^2 = (5-2)^2+(3-1)^2=13$  
4. We have two clusters: $k_1=\{a, c, d\}$ and $k_2=\{b, e\}$.
5. Calculate the centroids of the clusters $k_1$ and $k_2$:
 - $C_1 = (\frac{1+3+4}{3}, \frac{2+4+6}{3})=(\frac{8}{3}, 4)$.
 - $C_2 = (\frac{2+5}{2}, \frac{1+3}{2})=(\frac{7}{2}, 2)$.
6. Compute the distances from all points to the new centroids:
 - $d(a, C_1)^2 = (1-\frac{8}{3})^2 + (2-4)^2 = 6.78$
 - $d(a, C_2)^2 = (1-\frac{7}{2})^2 + (2-2)^2 = 6.25$
 - $d(b, C_1)^2 = (2-\frac{8}{3})^2 + (1-4)^2 = 9.43$
 - $d(b, C_2)^2 = (2-\frac{7}{2})^2 + (1-2)^2 = 3.25$
 - $d(c, C_1)^2 = (3-\frac{8}{3})^2 + (4-4)^2 = 0.11$
 - $d(c, C_2)^2 = (3-\frac{7}{2})^2 + (4-2)^2 = 4.25$
 - $d(d, C_1)^2 = (4-\frac{8}{3})^2 + (6-4)^2 = 5.78$
 - $d(d, C_2)^2 = (4-\frac{7}{2})^2 + (6-2)^2 = 16.25$
 - $d(e, C_1)^2 = (5-\frac{8}{3})^2 + (3-4)^2 = 6.44$
 - $d(e, C_2)^2 = (5-\frac{7}{2})^2 + (3-2)^2 = 3.25$
7. We have two new clusters: $k_1=\{c, d\}$ and $k_2=\{a, b, e\}$.
8. Recalculate the centroids of the clusters $k_1$ and $k_2$:
 - $C_1 = (\frac{3+4}{2}, \frac{4+6}{2})=(\frac{7}{2}, 5)$.
 - $C_2 = (\frac{1+2+5}{3}, \frac{2+1+3}{3})=(\frac{8}{3}, 2)$.
9. Compute the distances from all points to the new centroids:
 - $d(a, C_1)^2 = (1-\frac{7}{2})^2 + (2-5)^2 = 15.25$
 - $d(a, C_2)^2 = (1-\frac{8}{3})^2 + (2-2)^2 = 3.78$
 - $d(b, C_1)^2 = (2-\frac{7}{2})^2 + (1-5)^2 = 18.25$
 - $d(b, C_2)^2 = (2-\frac{8}{3})^2 + (1-2)^2 = 1.44$
 - $d(c, C_1)^2 = (3-\frac{7}{2})^2 + (4-5)^2 = 1.25$
 - $d(c, C_2)^2 = (3-\frac{8}{3})^2 + (4-2)^2 = 4.11$
 - $d(d, C_1)^2 = (4-\frac{7}{2})^2 + (6-5)^2 = 1.25$
 - $d(d, C_2)^2 = (4-\frac{8}{3})^2 + (6-2)^2 = 17.78$
 - $d(e, C_1)^2 = (5-\frac{7}{2})^2 + (3-5)^2 = 6.25$
 - $d(e, C_2)^2 = (5-\frac{8}{3})^2 + (3-2)^2 = 6.44$
10. We have two new clusters: $k_1=\{a, b\}$ and $k_2=\{c, d, e\}$.
11. Recalculate the centroids of the clusters $k_1$ and $k_2$:
 - $C_1 = (\frac{1+2}{2}, \frac{2+1}{2})=(\frac{3}{2}, \frac{3}{2})$.
 - $C_2 = (\frac{3+4+5}{3}, \frac{4+6+3}{3})=(4, \frac{13}{3})$.
12. Compute the distances from all points to the new centroids:
 - $d(a, C_1)^2 = (1-\frac{3}{2})^2 + (2-\frac{3}{2})^2 = 0.5$
 - $d(a, C_2)^2 = (1-4)^2 + (2-\frac{13}{3})^2 = 14.44$
 - $d(b, C_1)^2 = (2-\frac{3}{2})^2 + (1-\frac{3}{2})^2 = 0.5$
 - $d(b, C_2)^2 = (2-4)^2 + (1-\frac{13}{3})^2 = 15.11$
 - $d(c, C_1)^2 = (3-\frac{3}{2})^2 + (4-\frac{3}{2})^2 = 8.5$
 - $d(c, C_2)^2 = (3-4)^2 + (4-\frac{13}{3})^2 = 1.11$
 - $d(d, C_1)^2 = (4-\frac{3}{2})^2 + (6-\frac{3}{2})^2 = 26.5 $
 - $d(d, C_2)^2 = (4-4)^2 + (6-\frac{13}{3})^2 = 2.78$
 - $d(e, C_1)^2 = (5-\frac{3}{2})^2 + (3-\frac{3}{2})^2 = 14.5 $
 - $d(e, C_2)^2 = (5-4)^2 + (3-\frac{13}{3})^2 = 2.78$
13. The two clusters remain: $k_1=\{a, b\}$ and $k_2=\{c, d, e\}$.
14. The cluster assignments do not change. We are done.


```python
(5-3/2)**2+(3-3/2)**2
```

```python
(5-4)**2+(3-13/3)**2
```

## Should the number of clusters be selected beforehand for the k-means algorithm?
- A common challenge with *k*-means is that you must tell it how many clusters you expect: it cannot learn the number of clusters from the data.
- For example, if we ask the algorithm to identify six clusters, it will happily proceed and find the best six clusters:

```python
labels = KMeans(6, random_state=0, n_init=10).fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');
```

- Whether the result is meaningful is a question that is difficult to answer definitively; one approach that is rather intuitive, but that we won't discuss further here, is called [silhouette analysis](http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html).

- Alternatively, you might use a more complicated clustering algorithm which has a better quantitative measure of the fitness per number of clusters (e.g., Gaussian mixture models); or which *can* choose a suitable number of clusters (e.g., DBSCAN, mean-shift, or affinity propagation, all available in the ``sklearn.cluster`` submodule)

## Is the k-means algorithm limited to linear cluster boundaries?
- The fundamental model assumptions of *k*-means (points will be closer to their own cluster center than to others) means that the algorithm will often be ineffective if the clusters have complicated geometries.

- In particular, the boundaries between *k*-means clusters will always be linear, which means that it will fail for more complicated boundaries.
- Consider the following data, along with the cluster labels found by the typical *k*-means approach:

```python
from sklearn.datasets import make_moons
X, y = make_moons(200, noise=.05, random_state=0)
```

```python
labels = KMeans(2, random_state=0, n_init=10).fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');
```

- We can use a kernel transformation to project the data into a higher dimension where a linear separation is possible.

- One version of this kernelized *k*-means is implemented in Scikit-Learn within the ``SpectralClustering`` estimator.
- It uses the graph of nearest neighbors to compute a higher-dimensional representation of the data, and then assigns labels using a *k*-means algorithm:

```python
from sklearn.cluster import SpectralClustering
model = SpectralClustering(n_clusters=2, affinity='nearest_neighbors',
                           assign_labels='kmeans')
labels = model.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');
```

- We see that with this kernel transform approach, the kernelized *k*-means is able to find the more complicated nonlinear boundaries between clusters.

## Is the k-means algorithm slow for large numbers of samples?
- Because each iteration of *k*-means must access every point in the dataset, the algorithm can be relatively slow as the number of samples grows.
- You might wonder if this requirement to use all data at each iteration can be relaxed; for example, you might just use a subset of the data to update the cluster centers at each step.
- This is the idea behind batch-based *k*-means algorithms, one form of which is implemented in ``sklearn.cluster.MiniBatchKMeans``.
- The interface for this is the same as for standard ``KMeans``; we will see an example of its use as we continue our discussion.

## Examples

- Being careful about these limitations of the algorithm, we can use *k*-means to our advantage in a wide variety of situations.
We'll now take a look at a couple examples.

### Example 1: k-means on digits

- Here we will attempt to use *k*-means to try to identify similar digits *without using the original label information*; this might be similar to a first step in extracting meaning from a new dataset about which you don't have any *a priori* label information.

- We will start by loading the digits and then finding the ``KMeans`` clusters. The digits consist of 1,797 samples with 64 features, where each of the 64 features is the brightness of one pixel in an 8×8 image:

```python
from sklearn.datasets import load_digits
digits = load_digits()
digits.data.shape
```

- The clustering can be performed as we did before:

```python
kmeans = KMeans(n_clusters=10, random_state=0, n_init=10)
clusters = kmeans.fit_predict(digits.data)
kmeans.cluster_centers_.shape
```

- The result is 10 clusters in 64 dimensions.
- Notice that the cluster centers themselves are 64-dimensional points, and can themselves be interpreted as the "typical" digit within the cluster.
- Let's see what these cluster centers look like:

```python
fig, ax = plt.subplots(2, 5, figsize=(8, 3))
centers = kmeans.cluster_centers_.reshape(10, 8, 8)
for axi, center in zip(ax.flat, centers):
    axi.set(xticks=[], yticks=[])
    axi.imshow(center, interpolation='nearest', cmap=plt.cm.binary)
```

- We see that *even without the labels*, ``KMeans`` is able to find clusters whose centers are recognizable digits, with perhaps the exception of some digits.

- Because *k*-means knows nothing about the identity of the cluster, the 0–9 labels may be permuted.
- We can fix this by matching each learned cluster label with the true labels found in them:

```python
from scipy.stats import mode

labels = np.zeros_like(clusters)
for i in range(10):
    mask = (clusters == i)
    labels[mask] = mode(digits.target[mask])[0]
```

```python
fig, ax = plt.subplots(2, 5, figsize=(12, 9))
display_digits = digits.data[:10].reshape(10, 8, 8)
idx = 0
for axi, digi in zip(ax.flat, display_digits):
    axi.set(xticks=[], yticks=[])
    axi.imshow(digi, interpolation='nearest', cmap=plt.cm.binary)
    axi.set_xlabel("True Label:" + str(digits.target[idx]) + "\nPredicted Label:" + str(labels[idx]))
    idx += 1

plt.tight_layout()
```

- Now we can check how accurate our unsupervised clustering was in finding similar digits within the data:

```python
from sklearn.metrics import accuracy_score
accuracy_score(digits.target, labels)
```

With just a simple *k*-means algorithm, we discovered the correct grouping for 80% of the input digits!
Let's check the confusion matrix for this:

```python
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(digits.target, labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=digits.target_names,
            yticklabels=digits.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label');
```

- As we might expect from the cluster centers we visualized before, the main point of confusion is between the eights and ones.
- But this still shows that using *k*-means, we can essentially build a digit classifier *without reference to any known labels*!

- Just for fun, let's try to push this even farther.
We can use the t-distributed stochastic neighbor embedding (t-SNE) algorithm to pre-process the data before performing *k*-means.
- t-SNE is a nonlinear embedding algorithm that is particularly adept at preserving points within clusters.
Let's see how it does:

```python
from sklearn.manifold import TSNE

# Project the data: this step will take several seconds
tsne = TSNE(n_components=2, init='random', random_state=0)
digits_proj = tsne.fit_transform(digits.data)

# Compute the clusters
kmeans = KMeans(n_clusters=10, random_state=0)
clusters = kmeans.fit_predict(digits_proj)

# Permute the labels
labels = np.zeros_like(clusters)
for i in range(10):
    mask = (clusters == i)
    labels[mask] = mode(digits.target[mask])[0]

# Compute the accuracy
accuracy_score(digits.target, labels)
```

### That's nearly 94% classification accuracy *without using the labels*. This is the power of unsupervised learning when used carefully: it can extract information from the dataset that it might be difficult to do by hand or by eye.

### Example 2: *k*-means for color compression

- One interesting application of clustering is in color compression within images.
- For example, imagine you have an image with millions of colors.
- In most images, a large number of the colors will be unused, and many of the pixels in the image will have similar or even identical colors.

- For example, consider the image shown in the following figure, which is from the Scikit-Learn ``datasets`` module (for this to work, you'll have to have the ``pillow`` Python package installed).

```python
# Note: this requires the ``pillow`` package to be installed
from sklearn.datasets import load_sample_image
china = load_sample_image("china.jpg")
ax = plt.axes(xticks=[], yticks=[])
ax.imshow(china);
```

- The image itself is stored in a three-dimensional array of size ``(height, width, RGB)``, containing red/blue/green contributions as integers from 0 to 255:

```python
china.shape
```

- One way we can view this set of pixels is as a cloud of points in a three-dimensional color space.
- We will reshape the data to ``[n_samples x n_features]``, and rescale the colors so that they lie between 0 and 1:

```python
data = china / 255.0 # use 0...1 scale
data = data.reshape(427 * 640, 3)
data.shape
```

- We can visualize these pixels in this color space, using a subset of 10,000 pixels for efficiency:

```python
colors = data
rng = np.random.RandomState(0)
i = rng.permutation(data.shape[0])[:10000]
i.shape, i
```

```python
colors = colors[i]
colors
```

```python
R, G, B = data[i].T
R
```

```python
plt.scatter(R, G, color=colors, marker=".")
plt.xlabel("Red")
plt.ylabel("Green")
plt.title("16 Million Possible Colors")
```

```python
plt.scatter(R, B, color=colors, marker=".")
plt.xlabel("Red")
plt.ylabel("Blue")
plt.title("16 Million Possible Colors")
```

- Now let's reduce these 16 million colors to just 16 colors, using a *k*-means clustering across the pixel space.
- Because we are dealing with a very large dataset, we will use the mini batch *k*-means, which operates on subsets of the data to compute the result much more quickly than the standard *k*-means algorithm:

```python
from sklearn.cluster import MiniBatchKMeans
kmeans = MiniBatchKMeans(16)
kmeans.fit(data)
new_colors = kmeans.cluster_centers_[kmeans.predict(data)]
```

```python
new_colors.shape
```

```python
rng = np.random.RandomState(0)
i = rng.permutation(data.shape[0])[:10000]
colors = new_colors[i]
R, G, B = data[i].T
```

```python
plt.scatter(R, G, color=colors, marker=".")
plt.xlabel("Red")
plt.ylabel("Green")
plt.title("16 Possible Colors")
```

```python
plt.scatter(R, G, color=colors, marker=".")
plt.xlabel("Red")
plt.ylabel("Blue")
plt.title("16 Possible Colors")
```

- The result is a re-coloring of the original pixels, where each pixel is assigned the color of its closest cluster center.
- Plotting these new colors in the image space rather than the pixel space shows us the effect of this:

```python
china_recolored = new_colors.reshape(china.shape)

fig, ax = plt.subplots(1, 2, figsize=(16, 6),
                       subplot_kw=dict(xticks=[], yticks=[]))
fig.subplots_adjust(wspace=0.05)
ax[0].imshow(china)
ax[0].set_title('Original Image', size=16)
ax[1].imshow(china_recolored)
ax[1].set_title('16-color Image', size=16);
```

```python

```